In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pysabr import Hagan2002LognormalSABR
from pysabr import hagan_2002_lognormal_sabr as sabr
import xgboost as xgb
from math import sqrt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

In [ ]:
# template for using model and generated datasets

In [16]:
swaption=pd.read_csv("sabr.csv")
swaption['Date']=pd.to_datetime(swaption['Date'],format='%Y-%m-%d')
swaption=swaption.drop('Unnamed: 0',axis=1)
swaption_u=swaption.copy()
swaption_u=swaption_u.drop(['error','abs(e_size)>0.005'],axis=1)
swaption_u.set_index('Date',inplace=True)
# swaption_u
ir=pd.read_csv('../data/trade-price-ir-vegas.csv')
irdate=ir['Value Date'].drop_duplicates().reset_index(drop=True)
unique_combos=pd.read_csv('unique_combo.csv')
unique_combos=unique_combos.drop('Unnamed: 0',axis=1)

In [18]:
# calling dataframes with predicted sabr parameters and store it in dictionary datasets12 
datasets1 = {name: group for name, group in swaption_u.groupby(['Expiry', 'Tenor'])}
datasets=datasets1.copy()
datasets12={}
for name, group in datasets.items():
    filename = f"model/{name[0]}_{name[1]}.csv"
    df = pd.read_csv(filename)
    df['Date']=pd.to_datetime(df['Date'],format='%Y-%m-%d')
    df.set_index('Date',inplace=True)
    datasets12[name] = df

In [23]:
datasets12[1,1][datasets12[1,1].index=='2022-09-02']

,Expiry,Tenor,Strike,Vols,Forward Swap Rate,Numerical Strike,alpha,rho,volvol,sabrvol,...,dayofmonth,weekofyear,alphal1,alphal2,alphalrm2,alphal3,pre,prer,prev,vol
Date,,,,,,,,,,,,,,,,,,,,,
2022-09-02,1.0,1.0,atm-1.0%,0.183071,3.111863,3.080744,0.00177,0.231399,0.413552,0.180531,...,2,35,0.001797,0.001773,0.001785,0.001786,0.020717,0.330672,0.432894,1.189981


In [27]:
# calibrate vol when rate shock or new sabr parameters
def calibvol(strike,f_rate,exp,alpha,rho,volvol):
    f=f_rate
    strike=strike
    t=exp
    alpha=alpha
    beta=0.5
    rho=rho
    volvol=volvol
    calibrated_vols = sabr.lognormal_vol(strike, f, t, alpha, beta, rho, volvol) * 100 
    return calibrated_vols